In [38]:
import numpy as np
import pandas as pd


In [39]:
# read excel and geenrate dataframe
np.random.seed(3)
raw_dataset = pd.DataFrame(pd.read_excel(r'dataset_watermelon3.xlsx'))
#raw_dataset = raw_dataset.sample(frac = 1).reset_index(drop=True) # shuffle
raw_dataset.loc[raw_dataset['好瓜']=='是','好瓜']=1
raw_dataset.loc[raw_dataset['好瓜']=='否','好瓜']=0
print(raw_dataset)

# one-hot encoding
dataset = pd.DataFrame()
for col in raw_dataset.iloc[:,:-3].columns :
    for element in raw_dataset[col].unique():
        dataset[col+element]=np.zeros_like(raw_dataset[col])
        dataset.loc[raw_dataset[col]==element,col+element]=1
for col in raw_dataset.iloc[:,-3:].columns :
    dataset[col]=raw_dataset[col]

    
print(dataset)
dataset = dataset.as_matrix()[:,:-1]
print(dataset.shape)

    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率  好瓜
0   青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.460   1
1   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  0.774  0.376   1
2   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  0.634  0.264   1
3   青绿  蜷缩  沉闷  清晰  凹陷  硬滑  0.608  0.318   1
4   浅白  蜷缩  浊响  清晰  凹陷  硬滑  0.556  0.215   1
5   青绿  稍蜷  浊响  清晰  稍凹  软粘  0.403  0.237   1
6   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  0.481  0.149   1
7   乌黑  稍蜷  浊响  清晰  稍凹  硬滑  0.437  0.211   1
8   乌黑  稍蜷  沉闷  稍糊  稍凹  硬滑  0.666  0.091   0
9   青绿  硬挺  清脆  清晰  平坦  软粘  0.243  0.267   0
10  浅白  硬挺  清脆  模糊  平坦  硬滑  0.245  0.057   0
11  浅白  蜷缩  浊响  模糊  平坦  软粘  0.343  0.099   0
12  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  0.639  0.161   0
13  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  0.657  0.198   0
14  乌黑  稍蜷  浊响  清晰  稍凹  软粘  0.360  0.370   0
15  浅白  蜷缩  浊响  模糊  平坦  硬滑  0.593  0.042   0
16  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  0.719  0.103   0
    色泽青绿  色泽乌黑  色泽浅白  根蒂蜷缩  根蒂稍蜷  根蒂硬挺  敲声浊响  敲声沉闷  敲声清脆  纹理清晰  纹理稍糊  纹理模糊  \
0      1     0     0     1     0     0     1     0     0     1     0     0   
1      0     1     0     1     0  

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [54]:
def Euclidean_distance(class1, class2):
    '''
    input:numpy
    '''
    average_distance = []
    for i in range(class1.shape[0]):
        diff = class2 - class1[i,:]
        average_distance.append(((diff**2).sum(1)**0.5).mean())
    return np.mean(average_distance)

def Ncut_distance(class1,class2,sigma=1):
    def compute_W_matrix(samples,sigma=1):   
        num_of_samples = samples.shape[0]
        w = np.zeros((num_of_samples,num_of_samples))
        for sample1 in range(num_of_samples):
            for sample2 in range(num_of_samples):
                w[sample1,sample2] = np.exp(-((samples[sample1]-samples[sample2])**2).sum()/  \
                                              (2*sigma**2))
        return w
    
    def compute_D_matrix(W):
        D = np.zeros(W.shape)
        for i in range(W.shape[0]):
            D[i,i] = W[:,i].sum()
        return D
    
    def compute_Vol(D,Indi_vec):
        return (Indi_vec.T @ D).sum()
    
    def compute_Cut(W,Indi_vec):
        return Indi_vec.T @ W @(1-Indi_vec)
    
    all_samples = np.concatenate((class1,class2),axis=0)
    Indi_vec =np.concatenate( (np.ones((class1.shape[0],1)),
                    np.zeros((class2.shape[0],1))), axis=0  )

    W = compute_W_matrix(all_samples)
    D = compute_D_matrix(W)
    
    Vol_A = compute_Vol(D, Indi_vec)
    Vol_B = compute_Vol(D, 1-Indi_vec)
    Cut_AB = compute_Cut(W, Indi_vec)
    
    return Cut_AB/Vol_A + Cut_AB/Vol_B

def concatenate_two_categories(categories_list, method = 'Euclidean_distance'):
    if method=='Euclidean_distance' :
        shortest_distance = float("inf")
        for i,class1 in enumerate(categories_list):
            for j,class2 in enumerate(categories_list):
                # print(i,j)
                if i ==j :
                    continue
                distance  = Euclidean_distance(class1, class2)
                if distance < shortest_distance:
                    shortest_distance = distance
                    concatenate_index_1 = i
                    concatenate_index_2 = j
        new_class = np.concatenate((categories_list[concatenate_index_1],categories_list[concatenate_index_2]) ,axis=0)
        categories_list = [categories_list[i] for i in range(0, len(categories_list)) if i not in [concatenate_index_1, concatenate_index_2]]
        categories_list.append(new_class)
        print(new_class[:,-2:],'\n')
        return categories_list
    
    if method=='Ncut_distance' :
        shortest_distance = float("inf")
        for i,class1 in enumerate(categories_list):
            for j,class2 in enumerate(categories_list):
                # print(i,j)
                if i ==j :
                    continue
                distance  = Ncut_distance(class1, class2)
                if distance < shortest_distance:
                    shortest_distance = distance
                    concatenate_index_1 = i
                    concatenate_index_2 = j
        new_class = np.concatenate((categories_list[concatenate_index_1],categories_list[concatenate_index_2]) ,axis=0)
        categories_list = [categories_list[i] for i in range(0, len(categories_list)) if i not in [concatenate_index_1, concatenate_index_2]]
        categories_list.append(new_class)
        print(new_class[:,-2:],'\n')
        return categories_list

In [56]:
categories_list_Euc = [dataset[i][np.newaxis,...] for i in range( dataset.shape[0])]
print('Euclidean_distance:\n')
while len (categories_list_Euc) > 2:
    categories_list_Euc = concatenate_two_categories(categories_list_Euc,method='Euclidean_distance')


Euclidean_distance:

[[0.634 0.264]
 [0.556 0.215]] 

[[0.403 0.237]
 [0.36  0.37 ]] 

[[0.697 0.46 ]
 [0.608 0.318]] 

[[0.343 0.099]
 [0.593 0.042]] 

[[0.437 0.211]
 [0.403 0.237]
 [0.36  0.37 ]] 

[[0.774 0.376]
 [0.697 0.46 ]
 [0.608 0.318]] 

[[0.634 0.264]
 [0.556 0.215]
 [0.774 0.376]
 [0.697 0.46 ]
 [0.608 0.318]] 

[[0.481 0.149]
 [0.437 0.211]
 [0.403 0.237]
 [0.36  0.37 ]] 

[[0.639 0.161]
 [0.657 0.198]] 

[[0.666 0.091]
 [0.719 0.103]] 

[[0.245 0.057]
 [0.343 0.099]
 [0.593 0.042]] 

[[0.666 0.091]
 [0.719 0.103]
 [0.639 0.161]
 [0.657 0.198]] 

[[0.666 0.091]
 [0.719 0.103]
 [0.639 0.161]
 [0.657 0.198]
 [0.634 0.264]
 [0.556 0.215]
 [0.774 0.376]
 [0.697 0.46 ]
 [0.608 0.318]] 

[[0.481 0.149]
 [0.437 0.211]
 [0.403 0.237]
 [0.36  0.37 ]
 [0.666 0.091]
 [0.719 0.103]
 [0.639 0.161]
 [0.657 0.198]
 [0.634 0.264]
 [0.556 0.215]
 [0.774 0.376]
 [0.697 0.46 ]
 [0.608 0.318]] 

[[0.243 0.267]
 [0.245 0.057]
 [0.343 0.099]
 [0.593 0.042]] 



In [57]:
print('Ncut_distance:\n')
categories_list_Ncut = [dataset[i][np.newaxis,...] for i in range( dataset.shape[0])]
while len (categories_list_Ncut) > 2:
    categories_list_Ncut = concatenate_two_categories(categories_list_Ncut,method='Ncut_distance')

Ncut_distance:

[[0.666 0.091]
 [0.243 0.267]] 

[[0.245 0.057]
 [0.36  0.37 ]] 

[[0.608 0.318]
 [0.481 0.149]] 

[[0.774 0.376]
 [0.343 0.099]] 

[[0.403 0.237]
 [0.593 0.042]] 

[[0.719 0.103]
 [0.245 0.057]
 [0.36  0.37 ]] 

[[0.556 0.215]
 [0.666 0.091]
 [0.243 0.267]] 

[[0.697 0.46 ]
 [0.657 0.198]] 

[[0.639 0.161]
 [0.774 0.376]
 [0.343 0.099]] 

[[0.634 0.264]
 [0.719 0.103]
 [0.245 0.057]
 [0.36  0.37 ]] 

[[0.437 0.211]
 [0.697 0.46 ]
 [0.657 0.198]] 

[[0.608 0.318]
 [0.481 0.149]
 [0.403 0.237]
 [0.593 0.042]] 

[[0.556 0.215]
 [0.666 0.091]
 [0.243 0.267]
 [0.639 0.161]
 [0.774 0.376]
 [0.343 0.099]] 

[[0.437 0.211]
 [0.697 0.46 ]
 [0.657 0.198]
 [0.608 0.318]
 [0.481 0.149]
 [0.403 0.237]
 [0.593 0.042]] 

[[0.634 0.264]
 [0.719 0.103]
 [0.245 0.057]
 [0.36  0.37 ]
 [0.556 0.215]
 [0.666 0.091]
 [0.243 0.267]
 [0.639 0.161]
 [0.774 0.376]
 [0.343 0.099]] 

